In [17]:
# Get the data from google sheet and save as json
import requests
import csv
import json

url = "https://docs.google.com/spreadsheets/d/1W68sbH2vstHaWcKyfNG4uO7CaEumBAhqLZzgtjUFJhY/export?format=csv&gid=1651589789"
response = requests.get(url)
assert response.status_code == 200, 'Wrong status code'

csv_content = response.content.decode('utf-8')

# 2. Parse CSV content into a list of dictionaries
reader = csv.DictReader(csv_content.splitlines())
data_list = list(reader)

# 3. Convert list of dictionaries into JSON
json_data = json.dumps(data_list, indent=4)

# 4. Save JSON data to a file
with open('sheet.json', 'w') as json_file:
    json_file.write(json_data)

In [44]:
# take sheet.json and make fineprint html files
import json

with open('sheet.json') as json_file:
    data = json.load(json_file)
    
for row in data:
    text = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <title>Miss Information</title>
        <style>
            .container {{
                display: flex;
                flex-direction: column;
                align-items: center;
            }}

            body {{
                font-size: 1rem;
            }}
            
            button {{
                display: inline-block;
                padding: 8px 16px; /* ample padding for tappability */
                font-size: 14px; /* readable font size */
                font-weight: 500; /* semi-bold for better visibility */
                background-color: #d4adec; /* change to any color */
                color: #000000; /* white text */
                border: none;
                border-radius: 4px; /* rounded corners */
                box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* subtle shadow */
                transition: background-color 0.3s, transform 0.1s; /* smooth transition */
                text-align: center;
                cursor: pointer;
              }}
              
              /* Button hover state */
              button:hover {{
                background-color: #c697e3; /* slightly darker shade for hover */
              }}
              
              /* Button active state for feedback */
              button:active {{
                transform: scale(0.95); /* slightly scale down when pressed */
              }}
        </style>
    </head>
    <body>
    <h2>{question}</h2>
    <table >
        <tbody>
            <tr>
                <td>PRECISE:</td>
                <td>{range1}</td>
            </tr>
            <tr>
                <td>REFINED:</td>
                <td>{range2}</td>
            </tr>
            <tr>
                <td>ROUGH:</td>
                <td>{range3}</td>
            </tr>
        </tbody>
    </table>

    <br><br>

    <button id="toggleButton">SHOW ANSWER</button>

    <div id="myDiv" style="display: none;">
        <h2>{answer}</h2>
        <table >
            <tbody>
                <tr>
                    <td>PRECISE:</td>
                    <td>{interval1}</td>
                </tr>
                <tr>
                    <td>REFINED:</td>
                    <td>{interval2}</td>
                </tr>
                <tr>
                    <td>ROUGH:</td>
                    <td>{interval3}</td>
                </tr>
            </tbody>
        </table>
        <br>
    </div>
    
    <button onclick="random_question()" id="skipq">SKIP QUESTION</button>
    <button onclick="next_round()" id="next_round" style="display: none;">NEXT ROUND</button>

    <br><br>
    <h4>Fine Print</h4>
    {text}
    </body>

    </html>


    <script>
        document.getElementById("toggleButton").addEventListener("click", function() {{
            let div = document.getElementById("myDiv");
            if (div.style.display === "none") {{
                div.style.display = "block";
                this.style.display = "none";
                let skipq = document.getElementById("skipq");
                skipq.style.display = "none";
                let next_round = document.getElementById("next_round");
                next_round.style.display = "block";
            }} else {{
                div.style.display = "none";
                this.innerHTML = "Show Answer";
            }}
        }});
        
        
        // read the url params
        var queryString = window.location.search;
        var urlParams = new URLSearchParams(queryString);
        var num_players = urlParams.get('players');

        // if num_players exists and == 0 then remove buttons
        if (num_players == 0) {{
            let skipq = document.getElementById("skipq");
            skipq.style.display = "none";
            let next_round = document.getElementById("next_round");
            next_round.style.display = "none";
            let toggle_button = document.getElementById("toggleButton");
            toggle_button.style.display = "none";
        }}
        
        var q_data
        fetch('/sheet.json')
            .then(response => response.json())
            .then(data => {{
                q_data = data;
            }})
            .catch(error => {{
                console.error('Error fetching the JSON file:', error);
            }});
    
        function random_question() {{
            const randomIndex = Math.floor(Math.random() * q_data.length);
            const randomElement = q_data[randomIndex];
            window.location.href = "/questions/" + randomElement.id + ".html?players=" + num_players;
        }}   

        function next_round() {{
            window.location.href = "/index.html?players=" + num_players;

        }}   
    </script>
    """.format(question=row['question'], text=row['text'], range1=row['range1'], range2=row['range2'], range3=row['range3'], answer=row['answer'], interval1=row['interval1'], interval2=row['interval2'], interval3=row['interval3'])
    
    
    with open('questions/{id}.html'.format(id = row['id']), 'w') as html_file:
        html_file.write(text)

In [30]:
# make all the qr codes
import qrcode

with open('sheet.json') as json_file:
    data = json.load(json_file)
    
for row in data:
    url = 'https://missinformation.fun/questions/{id}.html?players=0'.format(id = row['id'])
    img = qrcode.make(url)
    img.save('images/qr_codes/{id}.png'.format(id = row['id']))